In [1]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)

    def __len__(self):
        return self._stop

    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [3]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma = 0.01):
        self.sigma = sigma
        
    def W(self, n_nodes1, n_nodes2):
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(1, n_nodes2)
        return B

In [4]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer, dropout_rate=0.5):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = initializer.W(n_nodes1, n_nodes2)
        self.B = initializer.B(n_nodes2)
        self.dZ = 0
        self.dA = 0
        self.dropout_rate = dropout_rate
        self.mask = None
        self.input_X_forward = 0
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """ 
        self.input_X_forward = X
        A = np.dot(X, self.W) + self.B
        
        return A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        
        dW = np.dot(self.input_X_forward.T, dA)
        dZ = np.dot(dA, self.W.T)
        self.dA = dA
        self.dW = dW
        self.dZ = dZ
        # 更新
        self = self.optimizer.update(self)
        return dZ

In [5]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス

        Returns
        ----------
        layer : 更新後の層のインスタンス
        """
        layer.B = layer.B - self.lr * np.average(layer.dA, axis=0)
        layer.W = layer.W - self.lr * layer.dW / layer.dA.shape[0]
        
        return layer

In [6]:
class Sigmoid:
    """
    活性化関数 : Sigmoid
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self):
        # 初期化
        self.input_X_forward = 0
    
    def _func(self, X):
        return 1 / (1 + np.exp(-1 * X))
    
    def _func_diff(self, X):
        return (1 - self._func(X)) * self._func(X)
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """ 
        self.input_X_forward = X
        A = self._func(X)
        return A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        
        grad = self._func_diff(self.input_X_forward)
        dZ = grad * dA
        return dZ

In [7]:
class Tanh:
    """
    活性化関数 : Sigmoid
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self):
        # 初期化
        self.input_X_forward = 0
    
    def _func(self, X):
        return np.tanh(X)
    
    def _func_diff(self, X):
        return 1 - (self._func(X))**2
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """ 
        self.input_X_forward = X
        A = self._func(X)
        return A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        
        grad = self._func_diff(self.input_X_forward)
        dZ = grad * dA
        return dZ

In [8]:
class softmax:
    """
    活性化関数 : Sigmoid
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self):
        # 初期化
        self.input_X_forward = 0
        self.pred = 0
    
    def _func(self, X):
        X = X - np.max(X)
        tmp = np.exp(X)
        denominator = np.sum(tmp, axis=1)
        output = tmp / denominator[:, np.newaxis]
        return output
    
    def _func_diff(self, X):
        return X
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """ 
        self.input_X_forward = X
        A = self._func(X)
        self.pred = A
        return A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        dZ = self.pred - dA
        
        return dZ

In [9]:
class ReLU:
    """
    活性化関数 : Sigmoid
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self):
        # 初期化
        self.input_X_forward = 0
    
    def _func(self, X):
        return np.maximum(0, X)
    
    def _func_diff(self, X):
        return np.where( x > 0, 1, 0)
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """ 
        self.input_X_forward = X
        A = self._func(X)
        return A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        
        grad = self._func_diff(self.input_X_forward)
        dZ = grad * dA
        return dZ

In [10]:
class XavierInitializer:
    """
    Xavierによる初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self):
        self.n_prev_nodes = 1
        pass
        
    def W(self, n_nodes1, n_nodes2):
        self.n_prev_nodes = n_nodes1
        W = np.random.randn(n_nodes1, n_nodes2) / np.sqrt(n_nodes1)
        return W
    
    def B(self, n_nodes2):
        B = np.random.randn(1, n_nodes2) / np.sqrt(self.n_prev_nodes)
        return B

In [11]:
class HeInitializer:
    """
    Heによる初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self):
        self.n_prev_nodes = 1
        pass
        
    def W(self, n_nodes1, n_nodes2):
        self.n_prev_nodes = n_nodes1
        W = np.random.randn(n_nodes1, n_nodes2) * np.sqrt(2 / n_nodes1)
        return W
    
    def B(self, n_nodes2):
        B = np.random.randn(1, n_nodes2) * np.sqrt(2 / self.n_prev_nodes)
        return B

In [12]:
class AdaGrad:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
        self.H_B = 1
        self.H_W = 1
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス

        Returns
        ----------
        layer : 更新後の層のインスタンス
        """
        
        #dA, dWを更新＆保存
        self.H_B = self.H_B + np.average(layer.dA)**2
        self.H_W = self.H_W + np.average(layer.dW)**2
        
        layer.B = layer.B - self.lr * np.average(layer.dA, axis=0) / np.sqrt(self.H_B)
        layer.W = layer.W - self.lr * layer.dW / layer.dA.shape[0] / np.sqrt(self.H_W)
        
        return layer

###### 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。
今回はバッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。
フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。

In [13]:
class SimpleRNN:
    """
    SimpleRNN
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, W_x, B_x, W_h, initializer, optimizer, activation):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        #self.W1 = initializer.W(n_wx_nodes1, n_wx_nodes2)
        #self.B1 = initializer.B(1)
        self.Wx = W_x
        self.Bx = B_x
        self.Wh = W_h
        self.dA = 0
        self.dW = 0
        self.W = 0
        self.B = 0
        self.input_X_forward = 0
        self.input_prev_ht_forward = 0
        self.activation = activation
        self.n_sequece = 0
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, sequence, feature)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, sequece, node)
            出力
        """ 
        self.input_X_forward = X
        self.n_sequece = X.shape[1]
        tmp_prev_h = np.zeros((X.shape[1]+1, X.shape[0], self.Wx.shape[1])) 
        self.input_prev_ht_forward = np.zeros((X.shape[0], X.shape[1], self.Wx.shape[1])) 
        y = np.zeros((X.shape[0], X.shape[1], self.Wx.shape[1]))
        tmp_y = np.zeros((X.shape[1], X.shape[0], self.Wx.shape[1])) 
        for i in range(self.n_sequece):
            Xt = X[:,i]
            #Xt:(batch, Feature)
            tmp = np.dot(Xt, self.Wx) + self.Bx + tmp_prev_h[i]
            #tmp:(batch, Node1)
            tmp_y[i] = self.activation.forward(tmp)
            #h_prev:(batch, node2)
            tmp_prev_h[i+1] = np.dot(tmp_y[i], self.Wh)
            
        self.input_prev_ht_forward = tmp_prev_h.transpose(1,0,2)
        y = tmp_y.transpose(1,0,2)
        return y
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, sequence, n_nodes)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, sequence, feature)
            前に流す勾配
        """
        dz = np.zeros_like(self.input_X_forward)
        tmp_dz = dz.transpose(1,0,2)
        
        loss_h = np.zeros((dA.shape[0], dA.shape[1]+1, dA.shape[2]))
        for i in reversed(range(self.n_sequece)):
            loss = dA[:,i,:] + loss_h[:,i,:]
            loss = self.activation.backward(loss) * loss
            
            #tの出力に対するバックプロパゲーション
            #input_x_forward:(batch_size, sequence, feature), loss:(batch, node)
            dW = np.dot(self.input_X_forward[:,i,:].T, loss)
            #dW : (feature, node)
            #dW = dW.sum(axis=0)
            tmp_dz[i] = np.dot(loss, self.Wx.T)
            self.dA = loss
            self.dW = dW
            self.W = self.Wx
            self.B = self.Bx
            self = self.optimizer.update(self)
            self.Wx = self.W
            self.Bx = self.B           
            
            #t-1の出力へのバックプロパゲーション
            #次のシーケンスへ渡すLoss
            #loss:(batch, node) Wh:(node, node)
            loss_h[:,i+1,:] = np.dot(loss, self.Wh.T)
            #whへのフィードバック
            self.dA = loss
            #self.input_prev_ht_forward : (batch, sequence, node1), loss:(batch, node)
            dW = np.dot(self.input_prev_ht_forward[:,i,:].T, loss)
            #dW:(sequence, node)
            #dW = dW.sum(axis=0)
            self.dW = dW
            self.W = self.Wh
            self.B = 0
            self = self.optimizer.update(self)
            self.Wh = self.W
        
        dz = tmp_dz.transpose(1,0,2)
        return dz

###### 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。
入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。
ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

In [14]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes))
b = np.array([1])

In [15]:
rnn = SimpleRNN(w_x, 1, w_h, initializer=SimpleInitializer(), optimizer=SGD(0.01), activation=Tanh())

In [16]:
h = rnn.forward(x)

In [17]:
h.shape

(1, 3, 4)

In [18]:
h[0,2]

array([0.79494228, 0.81839002, 0.83939649, 0.85584174])

正解：h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

###### 【問題3】（アドバンス課題）バックプロパゲーションの実装
バックプロパゲーションを実装します。
RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。

In [19]:
dA = np.array([[[0.01, 0.02, 0.03, 0.04], [0.01, 0.02, 0.03, 0.04], [0.01, 0.02, 0.03, 0.04]]])

In [20]:
rnn.backward(dA)

array([[[4.75883037e-05, 6.05642872e-05],
        [4.75883582e-05, 6.05643690e-05],
        [4.75884400e-05, 6.05644781e-05]]])

###### 【問題4】（アドバンス課題）データセットでの学習・推定
これまで使ってきたニューラルネットワークにSimpleRNNを組み込み学習させ、動くことを確認してください。
IMDB Review Dataset | Kaggle
映画レビューデータセットを使用します。ベクトル化を行い、作成したRNNに入力してください。

データ読み込み

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re
from scipy.sparse.linalg import norm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [22]:
imdb_data = pd.read_csv('./input/imdb_master.csv',encoding="latin-1")
imdb_data.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [23]:
imdb_data = imdb_data.drop(["Unnamed: 0","type","file"],axis=1)

In [24]:
imdb_data['sentiment'] = imdb_data['label'].map({"neg":0, "pos":1})

In [25]:
imdb_data = imdb_data.drop(["label"],axis=1)

In [26]:
imdb_data = imdb_data.dropna()

In [27]:
imdb_data = imdb_data[['sentiment','review']]

In [28]:
imdb_data.head()

,sentiment,review
0,0.0,Once again Mr. Costner has dragged out a movie...
1,0.0,This is an example of why the majority of acti...
2,0.0,"First of all I hate those moronic rappers, who..."
3,0.0,Not even the Beatles could write songs everyon...
4,0.0,Brass pictures (movies is not a fitting word f...


In [29]:
imdb_data_selected = imdb_data[0:10]

In [30]:
y_raw = imdb_data_selected["sentiment"].values

In [31]:
x_raw = imdb_data_selected["review"].values

テキストクリーニング

In [32]:
def remove_URL(text):
    cleaned = re.sub(r"(https?|ftp)(:\/\/[-_\.!~*\'()a-zA-Z0-9;\/?:\@&=\+\$,%#]+)", "" ,str(text))
    
    return cleaned

In [33]:
def remove_bracket(text):
    cleaned = re.sub("(?<=【).*?(?=】)", "" ,str(text))

    return cleaned 

In [34]:
def remove_bracket2(text):
    cleaned = re.sub("(?<=\<).*?(?=\>)", "" ,str(text))

    return cleaned 

In [35]:
def remove_emoji(text):
    cleaned = re.sub("(?<=:).*?(?=:)", "" ,str(text))

    return cleaned 

In [36]:
def normalize_number(text):
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text

In [37]:
def text_cleaning2(text):
    tmp = remove_URL(text)
    tmp = remove_bracket(tmp)
    tmp = remove_bracket2(tmp)
    tmp = remove_emoji(tmp)
    cleand = normalize_number(tmp)
    return cleand

In [38]:
X = []
for i in range(x_raw.shape[0]):
    X.append(text_cleaning2(x_raw[i]))

ベクトル化

In [39]:
def text_vectorizer(text):
    vectorizer = TfidfVectorizer(analyzer="word")
    vectorizer.fit(text)
    return vectorizer

In [40]:
vectorizer = text_vectorizer(X)

In [41]:
x_vect = vectorizer.transform(X)

In [42]:
x_train, x_val, y_train, y_val = train_test_split(x_vect, y_raw, test_size=0.4)

RNNで分類

In [43]:
class NN():
    """
    ディープニューラルネットワーク分類器

    Parameters
    ----------

    Attributes
    ----------
    """
    
    def __init__(self, n_epoch, batch_size, verbose = False):
        self.verbose = verbose
        self.batch_size = batch_size
        self.n_epoch = n_epoch
        self.loss = 0
        self.loss_val = 0
        self.activation_func = 0
        self.affine_func = 0
        self.n_layer = 0
        self.layer_instance = [0 for _ in range(64)]

        #各インスタンスを生成
        #initializerインスタンス
        
    def _crossentropy(self, y_pred, y):
        #クロスエントロピーを計算する
        INF_AVOIDANCE = 1e-8
        cross_entropy = -1 * y * np.log(y_pred + INF_AVOIDANCE)
        return np.sum(cross_entropy, axis=1)
    
    def add_layer(self, model):
        self.layer_instance[self.n_layer] = model
        self.n_layer += 1
        return
    
    def delet_all_layer(self):
        #add_layerでセットしたlayer情報を全てクリアする
        self.layer_instance[0:self.n_layer] = 0
        self.n_layer = 0
        
        return
    
    def fit(self, X, y, X_val=None, y_val=None):
        #lossの記録用の配列を用意
        self.loss = [[0 for i in range(X.shape[0])] for j in range(self.n_epoch)]
        self.loss_val = [[0 for i in range(X.shape[0])] for j in range(self.n_epoch)]
        
        print("Start learning")
        
        i = 0
        get_mini_batch = GetMiniBatch(x_train, y_train, self.batch_size)
        for epoch in range(self.n_epoch):
            loop_count = 0
            sum_loss = 0
            for mini_X_train, mini_y_train in get_mini_batch:
                X = mini_X_train
                #Forwardの計算
                for layer in range(self.n_layer):
                    X = self.layer_instance[layer].forward(X)
                
                #Loss計算
                sum_loss += self._crossentropy(X, mini_y_train)
                    
                #Backwardの計算
                dz = mini_y_train
                for layer in reversed(range(0, self.n_layer)):
                    dz = self.layer_instance[layer].backward(dz)
                
                loop_count += 1
                
            #Epoch毎のLoss計算結果表示
            self.loss[i] = sum_loss / loop_count
            if X_val is not None and y_val is not None:
                y_val_pred = self._predict(X_val)
                self.loss_val[i] = self._crossentropy(y_val_pred, y_val)
                
            if self.verbose:
                #verboseをTrueにした際は学習過程などを出力する
                print("Epoch:{} \n Loss:\n{} Loss(val):\n{}".format(i+1, self.loss[i], self.loss_val[i]))
                
            i +=1
            
        return
    
    def predict(self, X):
        #Forwardの計算
        for layer in range(self.n_layer):
            X = self.layer_instance[layer].forward(X)
        
        max_val = np.max(X, axis=1)
        mask = np.ones_like(X)
        X[X == max_val[:,np.newaxis]] = 1
        X[X != mask] = 0        
        
        return X

    def _predict(self, X):
        #Forwardの計算
        for layer in range(self.n_layer):
            X = self.layer_instance[layer].forward(X)
        
        return X

In [44]:
initializer=SimpleInitializer()

In [45]:
x_train.shape

(6, 794)

In [46]:
n_nodes = 3000
wx = initializer.W(x_train.shape[1], n_nodes)
wh = initializer.W(n_nodes, n_nodes)
B = initializer.B(1)

In [47]:
NN = NN(10, 2, verbose = True)

In [48]:
NN.add_layer(SimpleRNN(wx, B, wh, initializer=SimpleInitializer(), optimizer=SGD(0.01), activation=Tanh()))

In [49]:
NN.add_layer(FC(x_train.shape[0], 2, SimpleInitializer(), SGD(0.01)))

In [50]:
NN.add_layer(softmax())

In [51]:
NN.fit(x_train, y_train, x_val, y_val)

Start learning


SystemError: <built-in function isinstance> returned a result with an error set

映画のレビュー結果のPos/Neg分類をRNNモデルを使って試みた。
分散表現にBow+TFIDFを用いたため、入力データがSparse行列となってしまい、スクラッチで作成したNNモデルではエラーとなってしまい計算できなかった。（Word2Vecを試します）

感情分析では文書全体から分類を行うためRNNを用いるのは最適ではないかもしれない。（RNNの各シーケンス出力をSoftmax等の分類機の入力にした場合、単語の継ながりを計算する意味があまり無いように思える。またRNNの最後のシーケンス出力を用いた場合、最後の単語に結果が引っ張られてしまう。）